In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import datetime as dt
from IPython.display import HTML
import requests as r
import os
import time
from splinter import Browser
import re 

In [2]:
executable_path = {'executable_path': 'c:\\chromedrv\chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
browser.visit('http://www.google.com')

In [4]:
#Build a web application that scrapes various websites for data related to the Mission to Mars and displays the information in a single HTML page. 

#Create empty dictionary to save info
mars =  {}

### Mars News

In [45]:
#Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text. 
#Assign the text to variables that can be referenced later.

url="https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
browser.visit(url)
results=browser.html
soup_a=BeautifulSoup(results,'html.parser')

In [46]:
#pull title and content teaser from website
news_title=soup_a.find('div',class_='content_title').get_text()
news_paragraph=soup_a.find('div',class_='article_teaser_body').get_text()
mars["news_title"] = news_title
mars["news_p"] = news_paragraph

In [48]:
#mars

### Mars Space Images

In [42]:
#Visit the url for JPL Featured Space Image.
#Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called featured_image_url.

mars_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(mars_url)
time.sleep(2)
browser.find_by_id("full_image").click()
time.sleep(2)
browser.find_link_by_partial_text("more info").click()
time.sleep(2)
results=browser.html
soup_a=BeautifulSoup(results,'html.parser')
result= soup_a.find("figure",class_="lede")
link= result.a.img["src"]

C:\Users\zarre\anaconda3\envs\PythonData\lib\site-packages\splinter\driver\webdriver\__init__.py:490: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  warnings.warn(


In [43]:
#Save full  link and add to dictionary
feature_image_url = "https://www.jpl.nasa.gov" + link
mars["feature_image"] = feature_image_url

### Mars Weather

In [25]:
#Visit the Mars Weather twitter account and scrape the latest Mars weather tweet from the page. 
#Save the tweet text for the weather report as a variable

mars_twitter_url = "https://twitter.com/marswxreport?lang=en"
browser.visit(mars_twitter_url)
time.sleep(5)
browser.execute_script("window.scrollTo(0,-80000);")
time.sleep(3)
results=browser.html
soup_a=BeautifulSoup(results,'html.parser')
pattern = re.compile(r'InSight')
mars_weather = soup_a.find('span',text=pattern)
mars_weather.get_text()
mars["weather"] = mars_weather#.get_text()


In [22]:
#Test
#tweets=soup_a.find_all("div",attrs={'data-testid':'tweet'})
#tweets

[]

### Mars Facts

In [39]:
#Visit the Mars Facts webpage and use Pandas to scrape the table containing facts about the planet
#Use Pandas to convert the data to a HTML table string.

mars_fact_url = "https://space-facts.com/mars/"
browser.visit(mars_fact_url)
results=browser.html
soup_a=BeautifulSoup(results,'html.parser')

In [40]:
facts=pd.read_html(mars_fact_url)
mars_facts = facts[0]
mars_facts.columns = ["description","value"]
mars_facts.set_index("description",inplace=True)
mars_facts_table = mars_facts.to_html()
mars_facts_table = mars_facts_table.replace('\n','')
mars["facts"] = mars_facts_table

### Mars Hemispheres

In [3]:
#Visit the USGS Astrogeology to obtain high resolution images for each of Mar's hemispheres.
#Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. 
#Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.

mars_hem_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(mars_hem_url)
hemispheres_image_url = []

for i in range(4):
    hemispheres = {}
    browser.find_by_css("a.product-item h3")[i].click()
    results=browser.html
    soup=BeautifulSoup(results,'html.parser')
    title = soup.find("h2",class_="title").text
    url = soup.find("a",text = "Sample")["href"]
    hemispheres["title"] = title
    hemispheres["img_url"] = url
    hemispheres_image_url.append(hemispheres)
    browser.back()

hemispheres_image_url
mars["hemispheres"] = hemispheres_image_url


NameError: name 'mars' is not defined

In [49]:
mars

{'hemispheres': [{0: 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
  {1: 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
  {2: 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
  {3: 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}],
 'weather': <span class="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0">InSight sol 708 (2020-11-22) low -93.2ºC (-135.8ºF) high -8.2ºC (17.2ºF)
 winds from the W at 6.0 m/s (13.4 mph) gusting to 20.2 m/s (45.2 mph)
 pressure at 7.40 hPa</span>,
 'facts': '<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>value</th>    </tr>    <tr>      <th>description</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr> 